# O1v5-SONATA rewiring based on simplified models of connectivity
This is an auxiliary notebook for configuring/testing 1st..5th-order connectome rewiring, which

 - sets up connectome manipulations (rewiring)
 - writes .json config file for running rewiring using sbatch

<u>Requirements</u>: [connectome-manipulator](https://bbpgitlab.epfl.ch/conn/structural/connectome_manipulator) package (v0.0.5.dev1 or later; with v4 config format and `parallel-manipulator` entry point)

ℹ️ Related ticket: [[ACCS-49](https://bbpteam.epfl.ch/project/issues/browse/ACCS-49)] "Rewiring V5 connectome based on five simplified models of connectivity"

IMPORTANT: For running actual model building on BB5, use <code>sbatch run_rewiring.sh manip_config__[...].json</code>


In [1]:
# Initialization

""" Global imports """
import json
import os

In [4]:
def default_manip_config(circuit_config, seed=3210, N_split=None):
    """Generates a default manipulation config dict w/o any specific manipulation."""
    manip_config = {}
    manip_config['circuit_config'] = circuit_config
    manip_config['seed'] = seed
    if N_split is not None:
        manip_config['N_split_nodes'] = N_split
    return manip_config

def order_from_name(name):
    order_str = name.split("Order")[0]
    order = [int(i) for i in order_str if i.isdigit()]
    assert len(order) == 1, "ERROR: Order could not be determined!"
    return order[0]

def order_from_model_file(model_file):
    with open(model_file, "r") as f:
        model_dict = json.load(f)
    order = order_from_name(model_dict.get("model", ""))
    assert order > 0, "ERROR: Invalid model order!"
    return order

def add_manip_to_config(manip_config, amount_pct, prob_model_file, delay_model_file, props_model_file):
    """Adds a specific n-th order rewiring operation to the (v4) manipulation config dict (in-place)."""
    order = order_from_model_file(prob_model_file)
    assert 'manip' not in manip_config, "ERROR: Manipulation operation already specified!"
    manip_config['manip'] = {'name': f'ConnRewireOrder{order}EE{amount_pct}pct',
                             'fcts': [{'source': 'conn_rewiring',
                                       'sel_src': {'synapse_class': 'EXC'},
                                       'sel_dest': {'synapse_class': 'EXC'},
                                       'syn_class': 'EXC',
                                       'keep_indegree': False,
                                       'reuse_conns': False,
                                       'gen_method': 'duplicate_randomize', 
                                       'amount_pct': amount_pct,
                                       'model_config': {'prob_model_spec': {'file': prob_model_file},
                                                        'delay_model_spec': {'file': delay_model_file},
                                                        'props_model_spec': {'file': props_model_file}}}]}

def export_manip_config(manip_config, config_path, print_cmd=False, circuit_name=None, output_base_path=None, N_parallel=None):
    """Writes manipulation config to .json config file(s)."""
    if not isinstance(config_path, list):
        config_path = [config_path]

    fn = f'manip_config__{manip_config["manip"]["name"]}.json'
    for cpath in config_path:
        with open(os.path.join(cpath, fn), 'w') as f:
            json.dump(manip_config, f, indent=2)
        print(f"Config file {fn} written to {cpath}")

    if print_cmd:
        assert circuit_name is not None and output_base_path is not None and N_parallel is not None, \
            "ERROR: circuit_name/output_base_path/N_parallel required for printing launch command!"
        print()
        output_dir = os.path.join(output_base_path, circuit_name + f'__{manip_config["manip"]["name"]}')
        print_launch_cmd(cpath, fn, output_dir, N_parallel)

def print_launch_cmd(config_path, config_fn, output_dir, N_parallel):
    run_cmd = f'sbatch run_rewiring_parallel.sh "{config_fn}" "{output_dir}" {N_parallel}'
    print(f"# LAUNCH COMMAND: [DON'T LAUNCH FROM WITHIN ANOTHER ALLOCATION!]")
    print(f"cd {config_path}")
    print(run_cmd)


In [6]:
# Rewiring configuration
circuit_path = '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA'
circuit_name = os.path.split(circuit_path)[-1]
# circuit_config = os.path.join(circuit_path, 'sonata', 'circuit_config.json') # SONATA config (.json)
# blue_config = os.path.join('..', 'CircuitConfig_TC_BlobStim') # Optional; path rel. to path location of circuit config
circuit_config = os.path.join(circuit_path, 'sonata', 'circuit_config_tmp.json') # SONATA config (.json)  # TEMP edges file with afferent_center_x/y/z properties preliminarily added w/o validation, see [NSETM-1222]
blue_config = os.path.join('..', 'CircuitConfig_TC_BlobStim_tmp') # Optional; path rel. to path location of circuit config

output_base_path = '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits'
config_paths = ['../configs', '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/simplified_connectome_models/configs']  # Write to local and proj9 folder


In [7]:
# Model locations
models_path = os.path.join('/gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/simplified_connectome_models/model_building', circuit_name, 'model')  # Location of model files
prob_model_files = [os.path.join(models_path, f'ConnProb1stOrderO1v5EE.json'),
                    os.path.join(models_path, f'ConnProb2ndOrderO1v5EE.json'),
                    os.path.join(models_path, f'ConnProb3rdOrderO1v5EE.json'),
                    os.path.join(models_path, f'ConnProb4thOrderO1v5EE.json'),
                    os.path.join(models_path, f'ConnProb5thOrderO1v5EE.json')]
delay_model_file = os.path.join(models_path, f'DistDepDelayO1v5EE.json')
props_model_file = os.path.join(models_path, 'ConnPropsPerPathwayO1v5EE.json')


## Create rewiring configs
- Whole O1v5 connectome
- EXC-EXC connections only

In [12]:
# 1st-order rewiring
manip_config = default_manip_config(circuit_config, seed=3210)
add_manip_to_config(manip_config, 100, prob_model_files[0], delay_model_file, props_model_file)
# manip_config['manip']['name'] = manip_config['manip']['name'] + '__TEST4'
export_manip_config(manip_config, config_paths, print_cmd=True, circuit_name=circuit_name, output_base_path=output_base_path, N_parallel=100)

Config file manip_config__ConnRewireOrder1EE100pct.json written to ../configs
Config file manip_config__ConnRewireOrder1EE100pct.json written to /gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/simplified_connectome_models/configs

# LAUNCH COMMAND: [DON'T LAUNCH FROM WITHIN ANOTHER ALLOCATION!]
cd /gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/simplified_connectome_models/configs
sbatch run_rewiring_parallel.sh "manip_config__ConnRewireOrder1EE100pct.json" "/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnRewireOrder1EE100pct" 100


---
## Tools for post-processing

### Create BlueConfig

In [ ]:
# TODO

### Create bbp-workflow config

In [15]:
# TODO